# Cleanup
This notebook is provided to help you clean up any resources you have created by running through the example. 

In [55]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import json

role = get_execution_role()
sm = boto3.Session().client(service_name='sagemaker')

### Delete the Cloud Formation stack
This will delete all the resources created on your behalf using Cloud Formation.

OR just document that they should go to the CFN console and delete the stack

In [56]:
## boto3 cfn.delete_stack

### Delete feature groups

In [57]:
try:
    sm.delete_feature_group(FeatureGroupName='cc-agg-batch-fg') 
    print('deleted batch fg')
except:
    pass

try:
    sm.delete_feature_group(FeatureGroupName='cc-agg-fg') # use if needed to re-create
    print('deleted fg')
except:
    pass

deleted batch fg
deleted fg


In [58]:
sm.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'transaction-feature-group-02-12-52-50',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:105242341581:feature-group/transaction-feature-group-02-12-52-50',
   'CreationTime': datetime.datetime(2020, 12, 2, 13, 0, 43, 462000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'transaction-feature-group-01-19-18-28',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:105242341581:feature-group/transaction-feature-group-01-19-18-28',
   'CreationTime': datetime.datetime(2020, 12, 1, 19, 18, 45, 59000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'transaction-feature-group-01-12-29-08',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:105242341581:feature-group/transaction-feature-group-01-12-29-08',
   'CreationTime': datetime.datetime(2020, 12, 1, 12, 30, 53, 245000, tzinfo=tzlocal(

### Stop the KDA SQL App
or this will get handled by the CFN delete stack

In [59]:
import boto3
kda_client = boto3.client('kinesisanalytics')

try:
    kda_client.stop_application(ApplicationName='cc-agg-app')
except:
    pass

try:
    create_timestamp = kda_client.describe_application(
        ApplicationName='cc-agg-app')['ApplicationDetail']['CreateTimestamp']
    kda_client.delete_application(ApplicationName='tmp-cc-agg-app',
                                  CreateTimestamp=create_timestamp)
except:
    pass
print('Stopped and deleted the KDA SQL app')

Stopped and deleted the KDA SQL app


### Delete the KDA SQL App

In [60]:
try:
    create_timestamp = kda_client.describe_application(
        ApplicationName='cc-agg-app')['ApplicationDetail']['CreateTimestamp']
    response = kda_client.delete_application(ApplicationName='cc-agg-app',
                                  CreateTimestamp=create_timestamp)
    print('Deleted KDA SQL app')
except:
    print('FAILED to delete KDA sql app')

FAILED to delete KDA sql app


### Remove the trigger from Lambda

In [65]:
import boto3
lambda_client = boto3.client('lambda')
cf_client = boto3.client('cloudformation')

try:
    STACK_NAME = 'FeatureStoreStack-Test'
    outputs = cf_client.describe_stacks(StackName=STACK_NAME)['Stacks'][0]['Outputs']
    for o in outputs:
        if o['OutputKey'] == 'PredictLambdaFunctionARN':
            lambda_to_model_arn = o['OutputValue']
except:
    msg = f'CloudFormation stack {STACK_NAME} was not found. Please set the STACK_NAME properly and re-run this cell'
    sys.exit(ValueError(msg))
    
paginator = lambda_client.get_paginator('list_event_source_mappings')
mapping_iterator = paginator.paginate(FunctionName=lambda_to_model_arn)

for m in mapping_iterator:
    if len(m['EventSourceMappings']) > 0:
        uuid = m['EventSourceMappings'][0]['UUID']
        print(f'Deleting mapping: {uuid}...')
        lambda_client.delete_event_source_mapping(UUID=uuid)

### Delete the Kinesis data stream

In [62]:
kinesis_client = boto3.client('kinesis')
try:
    kinesis_client.delete_stream(StreamName='cc-stream')
except:
    pass
print('deleted Kinesis stream')

deleted Kinesis stream


### Delete the SageMaker endpoint

In [63]:
%store -r
try:
    sm.delete_endpoint(EndpointName=endpoint_name)
except:
    pass